In [10]:
import os
import sys

import pandas as pd
import numpy as np
from tqdm import tqdm
from ensemble_boxes import weighted_boxes_fusion as wbf

In [11]:
def xywhn2xyxy(norm_center_x, norm_center_y, norm_w, norm_h):
    norm_center_x = float(norm_center_x)
    norm_center_y = float(norm_center_y)
    norm_w = float(norm_w)
    norm_h = float(norm_h)
    x_min = float(0) if (norm_center_x - norm_w / 2) < 0 else (norm_center_x - norm_w / 2)
    y_min = float(0) if (norm_center_y - norm_h / 2) < 0 else (norm_center_y - norm_h / 2)
    x_max = float(1) if (norm_center_x + norm_w / 2) > 1 else (norm_center_x + norm_w / 2) 
    y_max = float(1) if (norm_center_y + norm_h / 2) > 1 else (norm_center_y + norm_h / 2)
    return (x_min, y_min, x_max, y_max)

def xyxyn2xywhn(bbox):
    x_min, y_min, x_max, y_max  = bbox
    norm_w = np.abs(x_max-x_min)
    norm_h = np.abs(y_max-y_min)
    norm_center_x = (x_max+x_min)/2
    norm_center_y = (y_max+y_min)/2
    return (norm_center_x, norm_center_y, norm_w, norm_h)

In [17]:
def make_df_list(results_path):
    df_list = []
    file_names = []

    for i, result in enumerate(os.listdir(results_path)):
        data = []
        if i == 0:
            file_names.extend(sorted(os.listdir(os.path.join(results_path, result))))
        for file_name in tqdm(file_names):
            file_path = os.path.join(results_path, result, file_name)
            with open(file_path, 'r') as file:
                for line in file:
                    label, norm_center_x, norm_center_y, norm_w, norm_h, cs = line.split()
                    x_min, y_min, x_max, y_max = xywhn2xyxy(norm_center_x, norm_center_y, norm_w, norm_h)
                    data.append([file_name, int(label), float(x_min), 
                                    float(y_min), float(x_max), float(y_max), float(cs)])
        df_list.append(pd.DataFrame(data, columns=["file_name", "label", "x_min", 
                                        "y_min", "x_max", "y_max", "cs"]))
    print(f"{len(os.listdir(results_path))} dataframes were created")
    return df_list, file_names

In [18]:
def weights_box_fusion(results_path, save_dir):
    
    df_list, file_names = make_df_list(results_path)

    for file_name in tqdm(file_names):
        boxes_list=[]
        scores_list=[]
        labels_list=[]

        for df in df_list:
            sub_df = df[df['file_name']==file_name]
            boxes_list.append(sub_df[['x_min', 'y_min', 'x_max', 'y_max']].apply(lambda row: [row['x_min'], 
                                                                                              row['y_min'], 
                                                                                              row['x_max'], 
                                                                                              row['y_max']], 
                                                                                              axis=1).tolist())
            scores_list.append(sub_df['cs'].tolist())
            labels_list.append(sub_df['label'].tolist())
        
        if any(box_list for box_list in boxes_list if box_list):
            boxes, scores, labels = wbf(
                boxes_list, scores_list, labels_list,
                weights=None, iou_thr=0.55, skip_box_thr=0.001
            )
        
        with open(os.path.join(save_dir, file_name),'w') as f:
            for box, cs, label in zip(boxes, scores, labels):
                norm_center_x, norm_center_y, norm_w, norm_h=xyxyn2xywhn(box)
                f.write("%d %lf %lf %lf %lf %lf\n"%(label, norm_center_x, norm_center_y, norm_w, norm_h, cs))

In [9]:
results_path = '/workspace/traffic_light/wbf_input'
save_dir = '/workspace/traffic_light/wbf_output/wbf_yolo'

os.makedirs(save_dir, exist_ok=True)
weights_box_fusion(results_path, save_dir)

100%|██████████| 13505/13505 [00:00<00:00, 24008.50it/s]


2 dataframes were created
['10000000.txt', '10000001.txt', '10000002.txt', '10000003.txt', '10000004.txt', '10000005.txt', '10000006.txt', '10000007.txt', '10000008.txt', '10000009.txt', '10000010.txt', '10000011.txt', '10000012.txt', '10000013.txt', '10000014.txt', '10000015.txt', '10000016.txt', '10000017.txt', '10000018.txt', '10000019.txt', '10000020.txt', '10000021.txt', '10000022.txt', '10000023.txt', '10000024.txt', '10000025.txt', '10000026.txt', '10000027.txt', '10000028.txt', '10000029.txt', '10000030.txt', '10000031.txt', '10000032.txt', '10000033.txt', '10000034.txt', '10000035.txt', '10000036.txt', '10000037.txt', '10000038.txt', '10000039.txt', '10000040.txt', '10000041.txt', '10000042.txt', '10000043.txt', '10000044.txt', '10000045.txt', '10000046.txt', '10000047.txt', '10000048.txt', '10000049.txt', '10000050.txt', '10000051.txt', '10000052.txt', '10000053.txt', '10000054.txt', '10000055.txt', '10000056.txt', '10000057.txt', '10000058.txt', '10000059.txt', '10000060.txt

 52%|█████▏    | 7011/13505 [02:41<02:56, 36.80it/s]